In [1]:
import pandas as pd
import geopandas as gp

In [2]:
df22 = pd.read_csv('./oh_2022_gen_prec/oh_2022_gen_prec.csv')

In [3]:
df20 = gp.read_file('./oh_vest_20/oh_vest_20.shp')

In [4]:
df20.shape

(8941, 11)

In [5]:
df22.shape

(8933, 259)

In [6]:
df20['ID'] = df20['COUNTYFP20'].str.zfill(3) + '-' + df20['PRECINCT20']
df22['COUNTYFP22'] = df22['COUNTYFP'].astype(str).str.zfill(3)
df22['ID'] = df22['COUNTYFP22'] + '-' + df22['PRECCODE']
merge_id = pd.merge(df20[['ID', 'COUNTYFP20','PRECINCT20', 'NAME20']], df22[['ID', 'COUNTYFP22', 'PRECCODE', 'PRECNAME', 'County']], on='ID', how='outer',indicator=True)

In [7]:
county_not_all_match = set(merge_id['COUNTYFP20'][(merge_id['_merge']!='both')])
county_all_match = set(merge_id['COUNTYFP20'])-county_not_all_match

In [8]:
county_not_all_match

{'007',
 '023',
 '029',
 '035',
 '043',
 '061',
 '073',
 '085',
 '093',
 '095',
 '101',
 '113',
 '123',
 '133',
 '151',
 nan}

In [9]:
county_all_match

{'001',
 '003',
 '005',
 '009',
 '011',
 '013',
 '015',
 '017',
 '019',
 '021',
 '025',
 '027',
 '031',
 '033',
 '037',
 '039',
 '041',
 '045',
 '047',
 '049',
 '051',
 '053',
 '055',
 '057',
 '059',
 '063',
 '065',
 '067',
 '069',
 '071',
 '075',
 '077',
 '079',
 '081',
 '083',
 '087',
 '089',
 '091',
 '097',
 '099',
 '103',
 '105',
 '107',
 '109',
 '111',
 '115',
 '117',
 '119',
 '121',
 '125',
 '127',
 '129',
 '131',
 '135',
 '137',
 '139',
 '141',
 '143',
 '145',
 '147',
 '149',
 '153',
 '155',
 '157',
 '159',
 '161',
 '163',
 '165',
 '167',
 '169',
 '171',
 '173',
 '175'}

In [10]:
df20['prec_counter_20']=1
df22['prec_counter_22']=1
gpby20 = df20.groupby(['COUNTYFP20']).sum().reset_index()
gpby22 = df22.groupby(['COUNTYFP22']).sum().reset_index()
gpby_merge = pd.merge(gpby20, gpby22, left_on='COUNTYFP20', right_on='COUNTYFP22', how = 'outer',indicator=True)

In [11]:
gpby_merge[['COUNTYFP20', 'COUNTYFP22', 'prec_counter_20', 'prec_counter_22']][gpby_merge['prec_counter_20']!= gpby_merge['prec_counter_22']]

,COUNTYFP20,COUNTYFP22,prec_counter_20,prec_counter_22
3,007,007,105,104
7,015,015,32,33
8,017,017,282,291
11,023,023,90,76
12,025,025,166,168
14,029,029,87,73
17,035,035,976,975
20,041,041,165,167
21,043,043,63,62
30,061,061,563,562


In [12]:
fips_dict = pd.Series(df22['County'].values, index=df22['COUNTYFP22']).to_dict()

In [13]:
merge_id['COUNTYFP20'][(merge_id['_merge']!='both')].map(fips_dict).unique()

array(['Lorain', 'Ottawa', 'Hamilton', 'Lucas', 'Montgomery',
       'Columbiana', 'Erie', 'Clark', 'Lake', 'Cuyahoga', 'Marion',
       'Portage', 'Stark', 'Ashtabula', 'Hocking', nan], dtype=object)

In [14]:
merge_id['COUNTYFP20'][merge_id['COUNTYFP20'].isin(county_all_match)].map(fips_dict).unique()

array(['Athens', 'Union', 'Butler', 'Wood', 'Holmes', 'Fairfield',
       'Shelby', 'Logan', 'Knox', 'Scioto', 'Auglaize', 'Clermont',
       'Guernsey', 'Wyandot', 'Gallia', 'Crawford', 'Morgan', 'Monroe',
       'Madison', 'Williams', 'Ashland', 'Delaware', 'Huron', 'Jackson',
       'Washington', 'Trumbull', 'Highland', 'Noble', 'Ross', 'Clinton',
       'Preble', 'Medina', 'Fayette', 'Defiance', 'Miami', 'Van Wert',
       'Mahoning', 'Coshocton', 'Pickaway', 'Hardin', 'Putnam', 'Carroll',
       'Meigs', 'Lawrence', 'Geauga', 'Darke', 'Fulton', 'Wayne',
       'Belmont', 'Morrow', 'Adams', 'Brown', 'Henry', 'Muskingum',
       'Hancock', 'Sandusky', 'Warren', 'Licking', 'Seneca', 'Richland',
       'Summit', 'Pike', 'Allen', 'Franklin', 'Perry', 'Champaign',
       'Harrison', 'Paulding', 'Tuscarawas', 'Mercer', 'Jefferson',
       'Greene', 'Vinton'], dtype=object)